In [ ]:
class ExpressionTree(LinkedBinaryTree):
    """
    An arithmetic expression tree
    """

    def __init__(self, token, left = None, right= None) :
        super().__init__()
        if not isinstance(token,str):
            raise TypeError("Token must be a string")
        self._add_root(token)
        if left is not None:
            if token not in "+-*x/":
                raise ValueError("Token must be valid operator")
            self._attach(self.root(), left, right)
            
    def __str__(self): 
        """Return string representation of the expression"""
        pieces = []
        self._parenthesize_recu(self.root(), pieces)
        return " ".join(pieces)
    
    def _parenthesize_recur(self, p, result):
        """
        Agrega la representación de la subárbol de p a la lista de resultados.

        Args:
            p: Posición del nodo raíz del subárbol.
            result: Lista donde se agregarán las representaciones de los nodos del subárbol.

        Returns:
            None

        Descripción:
            Esta función agrega la representación de la subárbol de p a la lista de resultados. Si p es una hoja,
            se agrega el valor del nodo como una cadena a la lista de resultados. Si p no es una hoja, se agrega un
            paréntesis de apertura a la lista de resultados, luego se llama recursivamente a la función para agregar
            la representación del subárbol izquierdo de p a la lista de resultados, luego se agrega el operador del nodo
            a la lista de resultados y finalmente se llama recursivamente a la función para agregar la representación
            del subárbol derecho de p a la lista de resultados. Al final se agrega un paréntesis de cierre a la lista
            de resultados.

        """

        if self.is_leaf(p):
            result.append(str(p.element()))  # Valor de la hoja como una cadena
        else:
            result.append('(')  # Paréntesis de apertura
            self._parenthesize_recur(self.left(p), result)  # Subárbol izquierdo
            result.append(p.element())  # Operador del nodo
            self._parenthesize_recur(self.right(p), result)  # Subárbol derecho
            result.append(')')  # Paréntesis de cierre
    

    def evaluate(self):
        """
        Retorna el resultado numérico de la expresión.

        Args:
            None

        Returns:
            El resultado numérico de la expresión.

        Descripción:
            Esta función retorna el resultado numérico de la expresión evaluando el árbol de expresión. Llama a la
            función recursiva evaluate_recur para evaluar el subárbol con raíz en el nodo raíz del árbol completo.

        """
        return self.evaluate_recur(self.root())

    def _evaluate_recur(self, p):
        """
        Retorna el resultado numérico del subárbol con raíz en p.

        Args:
            p: Posición del nodo raíz del subárbol.

        Returns:
            El resultado numérico del subárbol.

        Descripción:
            Esta función retorna el resultado numérico del subárbol con raíz en p. Si p es una hoja, retorna el valor del
            nodo como un número flotante. Si p no es una hoja, se obtiene el operador del nodo y se evalúan recursivamente
            los subárboles izquierdo y derecho de p. Luego, se realiza la operación correspondiente según el operador y se
            retorna el resultado. Si el operador es '+' se suma el resultado de los subárboles izquierdo y derecho, si el
            operador es '-' se resta el resultado del subárbol derecho al resultado del subárbol izquierdo, si el operador
            es '/' se divide el resultado del subárbol izquierdo entre el resultado del subárbol derecho, y en cualquier otro
            caso se trata como una multiplicación, multiplicando el resultado del subárbol izquierdo por el resultado del
            subárbol derecho.

        """

        if self.is_leaf(p):
            return float(p.element())  # Valor de la hoja como un número flotante
        else:
            op = p.element()  # Operador del nodo
            left_val = self._evaluate_recur(self.left(p))  # Resultado del subárbol izquierdo
            right_val = self._evaluate_recur(self.right(p))  # Resultado del subárbol derecho

            if op == '+':
                return left_val + right_val
            elif op == '-':
                return left_val - right_val
            elif op == '/':
                return left_val / right_val
            else:
                return left_val * right_val 
            
    def build_expression_tree(tokens):
        """
        Retorna un ExpressionTree basado en una expresión tokenizada.

        Args:
            tokens: Lista de tokens que componen la expresión.

        Returns:
            Un ExpressionTree basado en la expresión tokenizada.

        Descripción:
            Esta función construye y retorna un ExpressionTree basado en una expresión tokenizada. Recorre los tokens
            de la expresión y los procesa de acuerdo a su tipo. Si un token es un operador (+, -, *, /), se apila en
            la lista S. Si un token es un valor literal, se crea un ExpressionTree trivial con ese valor y se apila en S.
            Si un token es un paréntesis derecho, se desapilan tres elementos de S: el subárbol derecho, el operador y
            el subárbol izquierdo. Luego se crea un nuevo ExpressionTree con esos elementos y se apila en S. Al finalizar,
            se retorna el ExpressionTree resultante.

        """

        S = []  # Usamos una lista en Python como pila
        for t in tokens:
            if t in '+-x*/':  # t es un símbolo de operador
                S.append(t)  # Apilar el símbolo de operador
            elif t not in '()':  # Considerar t como un valor literal
                S.append(ExpressionTree(t))  # Apilar un ExpressionTree trivial que almacena el valor
            elif t == ')':  # Componer un nuevo ExpressionTree a partir de tres partes constituyentes
                right = S.pop()  # Subárbol derecho de acuerdo a LIFO
                op = S.pop()  # Símbolo de operador
                left = S.pop()  # Subárbol izquierdo
                S.append(ExpressionTree(op, left, right))  # Apilar el nuevo ExpressionTree
            # Ignoramos un paréntesis izquierdo
        return S.pop()